# Task 08 - Support Vector Machines (SVM)
## Pattern Recognition and Machine Learning

Copy and import needed files/methods from previous assignment to this directory. 
Adding path to the previous assignment is not sufficient. Upload system
requires your code to be self contained.

In [ ]:
# uncomment following for interactive matplotlib
# %matplotlib notebook

from svm1 import *
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import copy
from PIL import Image

## init

In [ ]:
def unwrap(data):
    """
    Simple "hack" for preparing data from *.mat files
    """
    try:
        while (len(data) == 1) and (len(data.shape) > 0):
            data = data[0]
        for key in list(data.dtype.names):
            data[key] = unwrap(data[key])
    except:
        pass
    return data

def ndarray2dict(data, indexes=None):
    outputs = {}
    for key in list(data.dtype.names):
        value = unwrap(data[key])
        try:
            if len(value.shape) > 0:
                value = np.atleast_2d(value)
        except:
            pass
        outputs[key] = value
    if indexes is not None:
        for key in indexes:
            outputs[key] -= 1
    return outputs
    
data = scipy.io.loadmat("data_33rpz_svm_toy.mat")
X = np.atleast_2d(unwrap(data["X"]))
y = np.atleast_2d(unwrap(data["y"])).astype(np.int32)
y[y==2] = -1

## Train SVM on simple data

In [ ]:
X = np.array([[1, 2, 1, -1, -1, -2], [1, 1, 2, -1, -2, -1]])
y = np.array([[1, 1, 1, -1, -1, -1]])
C = 1;

[w, b, sv_idx]  = my_svm(X, y, C)

print(w)
print(b)
print(sv_idx)float('inf')

## Toy data

### Load the data

In [ ]:
data = scipy.io.loadmat("data_33rpz_svm_toy.mat")
X = np.atleast_2d(unwrap(data["X"]))
y = np.atleast_2d(unwrap(data["y"])).astype(np.int32)
y[y==2] = -1

### Train Soft-margin SVM

In [ ]:
C = 1
w, b, sv_idx = my_svm(X, y, C, options={'verb': False})

### Visualize

In [ ]:
plot_pts(X, y)
plot_boundary(plt.gca(), w, b, X[:, sv_idx])
plt.title('Soft-margin SVM, C = {}'.format(C))
plt.savefig('linear_svm.png')

## OCR data
### Load the data

In [ ]:
data = scipy.io.loadmat("data_33rpz_svm.mat")

def unpack_ocr_data(data):
    tst = unwrap(data["tst"])
    tst_images = tst[0]
    tst_labels = tst[1]

    tst = {'images': tst[0],
           'labels': tst[1]}

    trn = unwrap(data["trn"])
    trn_images = trn[0]
    trn_labels = trn[1]

    trn = {'images': trn[0],
           'labels': trn[1]}

    return tst, trn

tst, trn = unpack_ocr_data(data)
print(tst['labels'].shape)

X_trn, y_trn, norm_trn = compute_measurements_2d(trn)

### Find optimal C
This will probably take around 10 minutes

In [ ]:
np.random.seed(42)
trn_folds, tst_folds = crossval(y_trn.size, num_folds=10)

best_C = None
best_C_err = np.inf
for C in [10, 1, 0.1, 0.01, 0.001]:
    
    C_err = compute_test_error(trn_folds, tst_folds, X_trn, y_trn, C)
    
    if C_err < best_C_err:
        best_C_err = C_err
        best_C = C
    print('C {} -> err {}'.format(C, C_err))

print('best C: {}'.format(best_C))


### Train SVM with optimal C

In [ ]:
w, b, sv_idx = my_svm(X_trn, np.atleast_2d(y_trn), best_C, options={'verb': False})

### Classify test data

In [ ]:
X_tst, y_tst, norm_tst = compute_measurements_2d(tst, norm_trn)
classif = classif_lin_svm(X_tst, {'w': w, 'b': b})

In [ ]:
error = 42 # count it yourself
print(error)

### And visualize

In [ ]:
plt.figure()
plot_pts(X_tst, y_tst)
plot_boundary(plt.gca(), w, b, None)
plt.title('Linear SVM')
plt.savefig('ocr_svm_tst.png')

In [ ]:
labels = classif.copy()
labels[labels == 1] = 0
labels[labels == -1] = 1

labels = np.expand_dims(labels, axis=0)

plt.figure()
show_classification(tst['images'], labels, 'AC')
plt.savefig('ocr_svm_classif.png')
